In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

import picamera
from picamera import array
from picamera.array import PiMotionArray
import numpy as np

# Se crea el objeto Spotify
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
                                    client_secret="XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
                                    redirect_uri="http://polaris/callback",
                                    scope="user-read-private user-read-playback-state user-modify-playback-state"))
# Configuracion camara
WIDTH = 640
HEIGHT = 480
FRAMERATE = 24

# Se crea el objeto camara
camera = picamera.PiCamera()
camera.resolution = (WIDTH, HEIGHT)
camera.framerate = FRAMERATE

# Se definen las constantes necesarias para los motion vectors
max_flow = 640 / 16.0 * 480 / 16.0 * 2**7
flow_scale = 0.165
flow_coeff = 100 * flow_scale / max_flow

while True:    
    output = picamera.array.PiMotionArray(camera)
    # Se comienza a grabar
    camera.start_recording('/dev/null', format='h264', motion_output=output)
    # Wait de 1 segundos
    camera.wait_recording(1)
    # Se termina la grabacion
    camera.stop_recording()
    # Se calculan los vectores de movimiento
    y_motion = np.sum(output.array['y']) * flow_coeff
    if  y_motion > 0 and y_motion > 5:
        print("Izquierda ", y_motion)
        sp.previous_track()
    if y_motion < 0 and y_motion < -5:
        print("Derecha ", y_motion)
        sp.next_track()